In [13]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# the above code downloads a text file containing the entire works of shakespeare
# this is what we will feed into our model to train it, and hope that it will help to output natural shakespearean sounding language

--2023-06-15 14:45:22--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.08s   

2023-06-15 14:45:22 (13.5 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [14]:
# let's open the text file and see how many characters it has
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("Number of characters within our input dataset: ", len(text)) 

Number of characters within our input dataset:  1115394


In [15]:
#print the first 50 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [16]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
# this code prints the number of unique characters in our input data


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [17]:
# We are now goign to create a mapping from integers to characters
chars = sorted(list(set(text))) # the list of every unique character in the text
stoi = { character:index for index,character in enumerate(chars) } # Converts a string to integer
itos = { index:character for index,character in enumerate(chars) } # Converts an integer to string
encode = lambda string: [stoi[character] for character in string]  # Convert a string to a list of integers
decode = lambda list: ''.join([itos[index] for index in list])  # Convert a list of integers to a string
print(encode("We love casting spells."))
print(decode(encode("We love casting spells.")))

# Using characters as our tokens isn't a bad idea, but it might be smarter to consider 3 characters at a time etc
# This is because we could represent our sentence in a much smaller list, just with integers up to 5000 instead of 65 for example
# There's a obviously an optimal combination, but to keep things simple we're going to use a character tokenizer

[35, 43, 1, 50, 53, 60, 43, 1, 41, 39, 57, 58, 47, 52, 45, 1, 57, 54, 43, 50, 50, 57, 8]
We love casting spells.


In [18]:
# We are going to now encode the entire shakespeare text dataset, and turn it into a torch.Tensor
import torch # A torch.Tensor is a multi-dimensional matrix containing elements of a single data type.
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:15])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0])


In [20]:
# Now let's split this data into a training and validation set
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [21]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [23]:
x = train_data[:block_size] # this is the input to the transformer
y = train_data[1:block_size+1] # this is the target for the transformer
for t in range(block_size): # we iterate through the input, so the transformer can predict the next text for sizes from context sizes of 1 character to block_size
    context = x[:t+1] 
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [24]:
# as we sample the chunks of text, we want to have multiple batches of chunks processing in parallel (as we're using GPUs)
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('outputs:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 5

In [32]:
print(xb) # this is our input to the transformer

import torch
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Each input number will go to a specific row and will read off the logits (scores) for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # we take the index and pass them into the embedding table
        # we arrange this into a batch (4) by time (8) by channel (vocab_size, 65) tensor
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None: 
            loss = None
        else:
            # reshape our logits so we can pass in the logits in the way the cross entropy function EXPECTS
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            
            # get the negative log likelihood loss (cross entropy loss) from our predicted next characters (logits) and our targets
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # given an array of indices idx of size B (batch_size) by T
        # we want to generate the next n tokens defined by max_new_tokens
        for _ in range(max_new_tokens):
            # Get the predicted tokens
            logits, loss = self(idx)
            # Get the last element in the time dimension
            logits = logits[:, -1, :]
            # Use softmax to get our probabilities (B, C)
            probs = F.softmax(logits, dim=-1)
            # get 1 sample from the distribution for each batch
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append our next sampled index into the current sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) # call our model and pass in the inputs and outputs
print(logits.shape)
print(loss)

# create a tensor of size 1x1 of just a 0, a 0 is how we kick off a generation (it is the newline generator)
idx = torch.zeros((1,1), dtype=torch.long)
# we ask for 100 tokens from our model's generator function
# we convert this tensor into a list
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))
# Without any training, this will generate a random garble of letters and symbols

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3
